In [15]:
# !pip install -r ./requirements.txt

In [2]:
# !pip uninstall virny -y

In [4]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@feature/prepare_experiments_for_inprocessors

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@feature/prepare_experiments_for_inprocessors

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../../../..")

print('Current location: ', os.getcwd())

Current location:  /home/dh3553/projects/fairness-variance


## Import dependencies

In [4]:
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import ACSPublicCoverageDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS

from source.experiment_interface import run_exp_iter_with_inprocessor

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'


## Define Input Variables

In [5]:
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'ADB_acs_pubcov'
DB_COLLECTION_NAME = 'one_repair_lvl_many_models'
FAIRNESS_INTERVENTION_NAME = 'ADB'
FAIR_INTERVENTION_PARAMS_LST = ['debiased_classifier']
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                     FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME)

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', 'diff_fairness_interventions_exp',
                                FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, 'folk_CA_2018_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [6]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [7]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [8]:
import uuid

custom_table_fields_dct = {
#     'session_uuid': str(uuid.uuid4()),
    'session_uuid': 'bf843ff8-62e9-4aac-83bc-d805e3299fdc',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  bf843ff8-62e9-4aac-83bc-d805e3299fdc


## Initialize custom objects

In [9]:
data_loader = ACSPublicCoverageDataset(state=['CA'], year=2018, with_nulls=False,
                                       subsample_size=15_000, subsample_seed=42)
data_loader.X_data.head()

,SCHL,MAR,SEX,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,ESR,ST,FER,RAC1P,AGEP,PINCP
0,19,5,1,2,0,1,3,4,1,1,2,2,2,6,6,0,1,21,3150.0
1,16,5,1,2,0,3,3,4,4,1,2,2,2,1,6,0,9,18,1600.0
2,13,5,2,2,1,1,1,0,2,1,2,2,2,6,6,2,1,16,0.0
3,20,1,2,2,0,4,1,4,1,2,2,2,2,6,6,2,8,43,0.0
4,16,1,2,2,0,4,1,4,1,2,2,2,2,6,6,0,6,54,0.0


In [10]:
data_loader.X_data.shape

(15000, 19)

## Run experiment iterations

### Experiment iteration 1

In [11]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [ ]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='ACSPublicCoverageDataset',
                              inprocessor_name='AdversarialDebiasing',
                              verbose=True)

2024-01-08 14:47:01 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': "['debiased_classifier']",
 'model_init_seed': 100,
 'session_uuid': '84eeb5f0-4ebe-4d9f-94ef-53ae302c2264'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-08 14:47:01 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


intervention_option:  debiased_classifier
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__SCHL_1', 'cat__SCHL_10', 'cat__SCHL_11', 'cat__SCHL_12',
       'cat__SCHL_13', 'cat__SCHL_14', 'cat__SCHL_15', 'cat__SCHL_16',
       'cat__SCHL_17', 'cat__SCHL_18',
       ...
       'cat__RELP_3', 'cat__RELP_4', 'cat__RELP_5', 'cat__RELP_6',
       'cat__RELP_7', 'cat__RELP_8', 'cat__RELP_9', 'num__AGEP', 'num__WKHP',
       'SEX&RAC1P_binary'],
      dtype='object', length=724)
Top indexes of an X_test in the current base flow dataset:  Int64Index([10155, 11689, 12599, 12193,  8678,  8217,  4670, 12087,  5235,
             4189,  7278, 10642,  5284,  7002, 14642, 10594,  7701,  8686,
             8665,  6253],
           dtype='int64')
Top indexes of an y_test in the current base flow dataset:  Int64Index([10155, 11689, 12599, 12193,  8678,  8217,  4670, 12087,  5235,
             4189,  7278, 10642,  5284,  7002, 14642, 10594,  7701,  8686,
             8665,  6253],
           dtype

...

epoch 0; iter: 0; batch classifier loss: 0.709423; batch adversarial loss: 0.826485
epoch 1; iter: 0; batch classifier loss: 0.589269; batch adversarial loss: 0.774333
epoch 2; iter: 0; batch classifier loss: 0.757826; batch adversarial loss: 0.757554
epoch 3; iter: 0; batch classifier loss: 0.651808; batch adversarial loss: 0.683050
epoch 4; iter: 0; batch classifier loss: 0.559073; batch adversarial loss: 0.616526
epoch 5; iter: 0; batch classifier loss: 0.383718; batch adversarial loss: 0.587390
epoch 6; iter: 0; batch classifier loss: 0.361682; batch adversarial loss: 0.608442
epoch 7; iter: 0; batch classifier loss: 0.300812; batch adversarial loss: 0.533616
epoch 8; iter: 0; batch classifier loss: 0.428041; batch adversarial loss: 0.522132
epoch 9; iter: 0; batch classifier loss: 0.334713; batch adversarial loss: 0.554177
epoch 10; iter: 0; batch classifier loss: 0.337976; batch adversarial loss: 0.478305
epoch 11; iter: 0; batch classifier loss: 0.403465; batch adversarial loss:

2024-01-08 14:49:25.594013: W tensorflow/c/c_api.cc:304] Operation '{name:'04a4482e-ae24-11ee-be98-ef9b34f2853b/04a4482e-ae24-11ee-be98-ef9b34f2853b/adversary_model/b2/Adam_1/Assign' id:3205 op device:{requested: '', assigned: ''} def:{{{node 04a4482e-ae24-11ee-be98-ef9b34f2853b/04a4482e-ae24-11ee-be98-ef9b34f2853b/adversary_model/b2/Adam_1/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](04a4482e-ae24-11ee-be98-ef9b34f2853b/04a4482e-ae24-11ee-be98-ef9b34f2853b/adversary_model/b2/Adam_1, 04a4482e-ae24-11ee-be98-ef9b34f2853b/04a4482e-ae24-11ee-be98-ef9b34f2853b/adversary_model/b2/Adam_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


epoch 0; iter: 0; batch classifier loss: 0.683073; batch adversarial loss: 0.620856
epoch 1; iter: 0; batch classifier loss: 0.440653; batch adversarial loss: 0.621911
epoch 2; iter: 0; batch classifier loss: 0.365344; batch adversarial loss: 0.600834
epoch 3; iter: 0; batch classifier loss: 0.300769; batch adversarial loss: 0.601682
epoch 4; iter: 0; batch classifier loss: 0.272899; batch adversarial loss: 0.534356
epoch 5; iter: 0; batch classifier loss: 0.387933; batch adversarial loss: 0.513569
epoch 6; iter: 0; batch classifier loss: 0.376173; batch adversarial loss: 0.530133
epoch 7; iter: 0; batch classifier loss: 0.369582; batch adversarial loss: 0.561520
epoch 8; iter: 0; batch classifier loss: 0.275634; batch adversarial loss: 0.532561
epoch 9; iter: 0; batch classifier loss: 0.282988; batch adversarial loss: 0.506175
epoch 10; iter: 0; batch classifier loss: 0.250687; batch adversarial loss: 0.404790
epoch 11; iter: 0; batch classifier loss: 0.310366; batch adversarial loss:

2024-01-08 14:50:02.788655: W tensorflow/c/c_api.cc:304] Operation '{name:'04a448ba-ae24-11ee-be98-ef9b34f2853b/04a448ba-ae24-11ee-be98-ef9b34f2853b/adversary_model/b2/Adam_1/Assign' id:4012 op device:{requested: '', assigned: ''} def:{{{node 04a448ba-ae24-11ee-be98-ef9b34f2853b/04a448ba-ae24-11ee-be98-ef9b34f2853b/adversary_model/b2/Adam_1/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](04a448ba-ae24-11ee-be98-ef9b34f2853b/04a448ba-ae24-11ee-be98-ef9b34f2853b/adversary_model/b2/Adam_1, 04a448ba-ae24-11ee-be98-ef9b34f2853b/04a448ba-ae24-11ee-be98-ef9b34f2853b/adversary_model/b2/Adam_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


epoch 0; iter: 0; batch classifier loss: 0.688715; batch adversarial loss: 0.983039
epoch 1; iter: 0; batch classifier loss: 0.628121; batch adversarial loss: 1.100539
epoch 2; iter: 0; batch classifier loss: 0.900819; batch adversarial loss: 1.126426
epoch 3; iter: 0; batch classifier loss: 1.017392; batch adversarial loss: 1.014833
epoch 4; iter: 0; batch classifier loss: 0.997079; batch adversarial loss: 0.934127
epoch 5; iter: 0; batch classifier loss: 0.972972; batch adversarial loss: 0.840362
epoch 6; iter: 0; batch classifier loss: 1.035811; batch adversarial loss: 0.765038
epoch 7; iter: 0; batch classifier loss: 0.989932; batch adversarial loss: 0.694059
epoch 8; iter: 0; batch classifier loss: 1.016223; batch adversarial loss: 0.648634
epoch 9; iter: 0; batch classifier loss: 0.816278; batch adversarial loss: 0.583646
epoch 10; iter: 0; batch classifier loss: 0.814806; batch adversarial loss: 0.581283
epoch 11; iter: 0; batch classifier loss: 0.388503; batch adversarial loss:

### Experiment iteration 2

In [11]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [ ]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='ACSPublicCoverageDataset',
                              inprocessor_name='AdversarialDebiasing',
                              verbose=True)

2024-01-08 21:56:06 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': "['debiased_classifier']",
 'model_init_seed': 200,
 'session_uuid': 'bf843ff8-62e9-4aac-83bc-d805e3299fdc'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-08 21:56:06 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


intervention_option:  debiased_classifier
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__SCHL_1', 'cat__SCHL_10', 'cat__SCHL_11', 'cat__SCHL_12',
       'cat__SCHL_13', 'cat__SCHL_14', 'cat__SCHL_15', 'cat__SCHL_16',
       'cat__SCHL_17', 'cat__SCHL_18', 'cat__SCHL_19', 'cat__SCHL_2',
       'cat__SCHL_20', 'cat__SCHL_21', 'cat__SCHL_22', 'cat__SCHL_23',
       'cat__SCHL_24', 'cat__SCHL_3', 'cat__SCHL_4', 'cat__SCHL_5',
       'cat__SCHL_6', 'cat__SCHL_7', 'cat__SCHL_8', 'cat__SCHL_9',
       'cat__MAR_1', 'cat__MAR_2', 'cat__MAR_3', 'cat__MAR_4', 'cat__MAR_5',
       'cat__DIS_1', 'cat__DIS_2', 'cat__ESP_0', 'cat__ESP_1', 'cat__ESP_2',
       'cat__ESP_3', 'cat__ESP_4', 'cat__ESP_5', 'cat__ESP_6', 'cat__ESP_7',
       'cat__ESP_8', 'cat__CIT_1', 'cat__CIT_2', 'cat__CIT_3', 'cat__CIT_4',
       'cat__CIT_5', 'cat__MIG_1', 'cat__MIG_2', 'cat__MIG_3', 'cat__MIL_0',
       'cat__MIL_1', 'cat__MIL_2', 'cat__MIL_3', 'cat__MIL_4', 'cat__ANC_1',
       'cat__ANC_2', 'cat__ANC_3', 

Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 0.792746; batch adversarial loss: 1.005012
epoch 1; iter: 0; batch classifier loss: 0.949620; batch adversarial loss: 1.258500
epoch 2; iter: 0; batch classifier loss: 1.028043; batch adversarial loss: 1.144422
epoch 3; iter: 0; batch classifier loss: 1.045848; batch adversarial loss: 1.075771
epoch 4; iter: 0; batch classifier loss: 0.997219; batch adversarial loss: 0.978551
epoch 5; iter: 0; batch classifier loss: 0.916257; batch adversarial loss: 0.886703
epoch 6; iter: 0; batch classifier loss: 0.807742; batch adversarial loss: 0.777476
epoch 7; iter: 0; batch classifier loss: 0.756174; batch adversarial loss: 0.765084
epoch 8; iter: 0; batch classifier loss: 0.751323; batch adversarial loss: 0.715508
epoch 9; iter: 0; batch classifier loss: 0.661528; batch adversarial loss: 0.746465
epoch 10; iter: 0; batch classifier loss: 0.532497; batch adversarial loss: 0.583311
epoch 11; iter: 0; batch classifier loss: 0.556063; batch adversarial loss:

### Experiment iteration 3

In [11]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [12]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='ACSPublicCoverageDataset',
                              inprocessor_name='AdversarialDebiasing',
                              verbose=True)

2024-01-04 19:29:01 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 300,
 'experiment_iteration': 'Exp_iter_3',
 'fair_intervention_params_lst': '[True]',
 'model_init_seed': 300,
 'session_uuid': '0626d80f-e288-4f16-b8ab-260eb34d62d3'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-04 19:29:01 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-04 19:29:01 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_option:  True
Using ROC postprocessor
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
     

...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 4

In [13]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [14]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='ACSPublicCoverageDataset',
                              inprocessor_name='AdversarialDebiasing',
                              verbose=True)

2024-01-04 20:15:17 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 400,
 'experiment_iteration': 'Exp_iter_4',
 'fair_intervention_params_lst': '[True]',
 'intervention_param': 'True',
 'model_init_seed': 400,
 'session_uuid': '0626d80f-e288-4f16-b8ab-260eb34d62d3'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-04 20:15:17 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-04 20:15:17 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_option:  True
Using ROC postprocessor
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
     

...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 5

In [11]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [12]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='ACSPublicCoverageDataset',
                              inprocessor_name='AdversarialDebiasing',
                              verbose=True)

2024-01-08 21:57:45 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 500,
 'experiment_iteration': 'Exp_iter_5',
 'fair_intervention_params_lst': "['debiased_classifier']",
 'model_init_seed': 500,
 'session_uuid': 'bf843ff8-62e9-4aac-83bc-d805e3299fdc'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-08 21:57:45 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


intervention_option:  debiased_classifier
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__SCHL_1', 'cat__SCHL_10', 'cat__SCHL_11', 'cat__SCHL_12',
       'cat__SCHL_13', 'cat__SCHL_14', 'cat__SCHL_15', 'cat__SCHL_16',
       'cat__SCHL_17', 'cat__SCHL_18', 'cat__SCHL_19', 'cat__SCHL_2',
       'cat__SCHL_20', 'cat__SCHL_21', 'cat__SCHL_22', 'cat__SCHL_23',
       'cat__SCHL_24', 'cat__SCHL_3', 'cat__SCHL_4', 'cat__SCHL_5',
       'cat__SCHL_6', 'cat__SCHL_7', 'cat__SCHL_8', 'cat__SCHL_9',
       'cat__MAR_1', 'cat__MAR_2', 'cat__MAR_3', 'cat__MAR_4', 'cat__MAR_5',
       'cat__DIS_1', 'cat__DIS_2', 'cat__ESP_0', 'cat__ESP_1', 'cat__ESP_2',
       'cat__ESP_3', 'cat__ESP_4', 'cat__ESP_5', 'cat__ESP_6', 'cat__ESP_7',
       'cat__ESP_8', 'cat__CIT_1', 'cat__CIT_2', 'cat__CIT_3', 'cat__CIT_4',
       'cat__CIT_5', 'cat__MIG_1', 'cat__MIG_2', 'cat__MIG_3', 'cat__MIL_0',
       'cat__MIL_1', 'cat__MIL_2', 'cat__MIL_3', 'cat__MIL_4', 'cat__ANC_1',
       'cat__ANC_2', 'cat__ANC_3', 

...

epoch 128; iter: 0; batch classifier loss: 0.369092; batch adversarial loss: 0.487982
epoch 129; iter: 0; batch classifier loss: 0.388461; batch adversarial loss: 0.583552
epoch 130; iter: 0; batch classifier loss: 0.366600; batch adversarial loss: 0.578737
epoch 131; iter: 0; batch classifier loss: 0.359306; batch adversarial loss: 0.536123
epoch 132; iter: 0; batch classifier loss: 0.435319; batch adversarial loss: 0.515973
epoch 133; iter: 0; batch classifier loss: 0.357801; batch adversarial loss: 0.488801
epoch 134; iter: 0; batch classifier loss: 0.451620; batch adversarial loss: 0.571754
epoch 135; iter: 0; batch classifier loss: 0.476338; batch adversarial loss: 0.533202
epoch 136; iter: 0; batch classifier loss: 0.342289; batch adversarial loss: 0.571457
epoch 137; iter: 0; batch classifier loss: 0.366089; batch adversarial loss: 0.516475
epoch 138; iter: 0; batch classifier loss: 0.424633; batch adversarial loss: 0.507675
epoch 139; iter: 0; batch classifier loss: 0.425000; b

epoch 26; iter: 0; batch classifier loss: 0.479527; batch adversarial loss: 0.546547
epoch 27; iter: 0; batch classifier loss: 0.508064; batch adversarial loss: 0.578316
epoch 28; iter: 0; batch classifier loss: 0.493133; batch adversarial loss: 0.636589
epoch 29; iter: 0; batch classifier loss: 0.472463; batch adversarial loss: 0.523175
epoch 30; iter: 0; batch classifier loss: 0.494474; batch adversarial loss: 0.613815
epoch 31; iter: 0; batch classifier loss: 0.431302; batch adversarial loss: 0.519920
epoch 32; iter: 0; batch classifier loss: 0.469801; batch adversarial loss: 0.584210
epoch 33; iter: 0; batch classifier loss: 0.451620; batch adversarial loss: 0.538427
epoch 34; iter: 0; batch classifier loss: 0.489235; batch adversarial loss: 0.478884
epoch 35; iter: 0; batch classifier loss: 0.591743; batch adversarial loss: 0.572693
epoch 36; iter: 0; batch classifier loss: 0.432099; batch adversarial loss: 0.554630
epoch 37; iter: 0; batch classifier loss: 0.426311; batch adversa

epoch 125; iter: 0; batch classifier loss: 0.389145; batch adversarial loss: 0.516773
epoch 126; iter: 0; batch classifier loss: 0.338064; batch adversarial loss: 0.554114
epoch 127; iter: 0; batch classifier loss: 0.321693; batch adversarial loss: 0.477825
epoch 128; iter: 0; batch classifier loss: 0.286041; batch adversarial loss: 0.582287
epoch 129; iter: 0; batch classifier loss: 0.406036; batch adversarial loss: 0.534584
epoch 130; iter: 0; batch classifier loss: 0.336754; batch adversarial loss: 0.515961
epoch 131; iter: 0; batch classifier loss: 0.386196; batch adversarial loss: 0.524300
epoch 132; iter: 0; batch classifier loss: 0.374722; batch adversarial loss: 0.581973
epoch 133; iter: 0; batch classifier loss: 0.348025; batch adversarial loss: 0.571248
epoch 134; iter: 0; batch classifier loss: 0.281098; batch adversarial loss: 0.579927
epoch 135; iter: 0; batch classifier loss: 0.312052; batch adversarial loss: 0.600423
epoch 136; iter: 0; batch classifier loss: 0.434445; b

epoch 23; iter: 0; batch classifier loss: 0.539790; batch adversarial loss: 0.498482
epoch 24; iter: 0; batch classifier loss: 0.465084; batch adversarial loss: 0.571314
epoch 25; iter: 0; batch classifier loss: 0.500288; batch adversarial loss: 0.507718
epoch 26; iter: 0; batch classifier loss: 0.500003; batch adversarial loss: 0.507159
epoch 27; iter: 0; batch classifier loss: 0.493842; batch adversarial loss: 0.499384
epoch 28; iter: 0; batch classifier loss: 0.486675; batch adversarial loss: 0.497902
epoch 29; iter: 0; batch classifier loss: 0.438579; batch adversarial loss: 0.596211
epoch 30; iter: 0; batch classifier loss: 0.513032; batch adversarial loss: 0.494489
epoch 31; iter: 0; batch classifier loss: 0.485277; batch adversarial loss: 0.521206
epoch 32; iter: 0; batch classifier loss: 0.446753; batch adversarial loss: 0.489576
epoch 33; iter: 0; batch classifier loss: 0.437246; batch adversarial loss: 0.489248
epoch 34; iter: 0; batch classifier loss: 0.518548; batch adversa

epoch 122; iter: 0; batch classifier loss: 0.331774; batch adversarial loss: 0.525580
epoch 123; iter: 0; batch classifier loss: 0.366286; batch adversarial loss: 0.534477
epoch 124; iter: 0; batch classifier loss: 0.324732; batch adversarial loss: 0.520542
epoch 125; iter: 0; batch classifier loss: 0.397138; batch adversarial loss: 0.548304
epoch 126; iter: 0; batch classifier loss: 0.320344; batch adversarial loss: 0.495626
epoch 127; iter: 0; batch classifier loss: 0.447816; batch adversarial loss: 0.534401
epoch 128; iter: 0; batch classifier loss: 0.321174; batch adversarial loss: 0.574994
epoch 129; iter: 0; batch classifier loss: 0.443316; batch adversarial loss: 0.428621
epoch 130; iter: 0; batch classifier loss: 0.377852; batch adversarial loss: 0.585757
epoch 131; iter: 0; batch classifier loss: 0.350539; batch adversarial loss: 0.537072
epoch 132; iter: 0; batch classifier loss: 0.326016; batch adversarial loss: 0.512954
epoch 133; iter: 0; batch classifier loss: 0.380882; b

### Experiment iteration 6

In [13]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [14]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='ACSPublicCoverageDataset',
                              inprocessor_name='AdversarialDebiasing',
                              verbose=True)

2024-01-05 04:49:43 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 600,
 'experiment_iteration': 'Exp_iter_6',
 'fair_intervention_params_lst': '[True]',
 'intervention_param': 'True',
 'model_init_seed': 600,
 'session_uuid': '0626d80f-e288-4f16-b8ab-260eb34d62d3'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-05 04:49:43 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-05 04:49:43 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_option:  True
Using ROC postprocessor
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
     

...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]